In [ ]:
import soccerdata as sd
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm

In [ ]:
print(sd.WhoScored.available_leagues())

In [ ]:
# Arreglar : ---
# En serie
leagues = "ARG-PrimeraDivision"
seasons = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
ws = sd.WhoScored(leagues=leagues, seasons=seasons, no_cache=True)
epl_schedule = ws.read_schedule().reset_index()

game_ids = epl_schedule["game_id"]

trabajo = list(game_ids)
events = ws.read_events(match_id=game_ids)

In [ ]:
events.to_csv(f"WhoScored_{leagues}.csv", header=True)

In [ ]:
# Paralelo
league = "USA-MLS"
seasons = [i for i in range(2013, 2023)]

ws = sd.WhoScored(leagues=league, seasons=seasons, no_cache=True)
schedule = ws.read_schedule().reset_index()

game_ids = schedule["game_id"]

trabajo = list(game_ids)
workers = 30
division_work = []
tamaño_sublistas = len(trabajo) // workers
for n in range(workers):
    work = [trabajo[n*tamaño_sublistas+i] for i in range(tamaño_sublistas)]
    division_work.append(work)
# El resto
resto = len(trabajo) % workers
for i in range(resto):
    division_work[0].append(trabajo[-i-1])


def read_parallel(game_ids, leagues, seasons):
    try:
        ws = sd.WhoScored(leagues=leagues, seasons=seasons, no_cache=False)
        events = ws.read_events(match_id=game_ids)
        return events
    except:
        return pd.DataFrame()


n_workers = 10
resultado = Parallel(n_jobs=n_workers, verbose=10)(
    delayed(read_parallel)(w, league, seasons) for w in division_work)

In [ ]:
# Guardar
df = pd.DataFrame()
for i in resultado:
    df = pd.concat([df,i])
df = df.reset_index()
df.to_csv(f"WhoScored_{league}.csv", header=True)

In [ ]:
set_problematicos =[]
# Checkeo de que se guardó
for i in range(workers):
    print(f"Id: {i}, {len(resultado[i])}")
    if len(resultado[i]) == 0:
        set_problematicos.append(i)    

In [ ]:
resultado

In [ ]:
df

In [ ]:
# Solucionando errores
    
# Parte de la solución
# set_problematicos = #MANUAL
ids_con_problmas = [1076372,1739797,1748533,1742792]
problem_division_work = [list(set(division_work[i]) - set(ids_con_problmas)) for i in set_problematicos]

problem_division_work = [item for sublist in problem_division_work for item in sublist]
ws = sd.WhoScored(leagues=league, seasons=seasons, no_cache = True)
events = ws.read_events(match_id=problem_division_work)


In [ ]:
events

In [ ]:
df = pd.read_csv(f"WhoScored_{league}.csv")
events = events.reset_index()
df = pd.concat([df,events])

In [ ]:
df.to_csv(f"WhoScored_{league}.csv", header=True)

In [ ]:
# Antes de seguir manipulandolo, vemos que todo esté correcto
leagues_to_sort = ["ENG-Premier League", "ESP-La Liga",
                   "FRA-Ligue 1", "GER-Bundesliga", "ITA-Serie A"]

df = pd.read_csv(f"WhoScored_{leagues_to_sort[0]}.csv")

In [ ]:
leagues_to_sort = ["ESP-La Liga", "FRA-Ligue 1", "GER-Bundesliga", "ITA-Serie A"]

# Ordenar por fecha, id, minuto, segundo
for l in leagues_to_sort:
    print(l)
    df = pd.read_csv(f"WhoScored_{l}.csv")
    print(df.columns)
    # Configuración de fecha
    df['fecha'] = df['game'].str.slice(stop=10)
    fecha_column = df.pop('fecha')  # Extraemos la columna 'fecha'
    df.insert(0, 'fecha', fecha_column)  # Insertamos la columna 'fecha' al principio del DataFrame
    # Configuración de columnas
    df = df.drop(['Unnamed: 0'], axis= 1) # Para arreglar
    # Orden 
    df = df.sort_values(by=["fecha","game_id","minute","second"],ignore_index=True)
    df.to_csv(f"WhoScored_{l}.csv", header=True,index=False)


In [ ]:
df